In [110]:
import numpy as np 
import matplotlib.pyplot as plt 

cm = 1/2.54

%matplotlib qt
normalsize = 12
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Cambria",
    "font.size": normalsize
})


# 1D arrays 
W = 100.0
H = 100.0
x = np.arange(0,W+0.1,0.1) 
y = np.arange(-H,H+0.1,0.1) 
y[-1] = H

# Meshgrid 
X,Y = np.meshgrid(x,y)
print
  
# Assign vector directions 
Ch = 1
q = 0
Fa = -abs(Ch/X) - abs(0.6*(Y-q) / (1+abs(Y-q)) - 0.59)
# Fa = -abs(Ch/X) - abs(-1 / (1 + np.exp(Y-q)))
Fb = Y/X - Fa*X
Ftheta = np.arctan(2*Fa*X + Fb)

Ex = -np.cos(Ftheta)
Ey = -np.sin(Ftheta)

start_x = np.arange(0,H+10,10)

stream_points = np.column_stack((start_x, np.sqrt(H*H - start_x*start_x)))
stream_points = np.append(stream_points, stream_points*[1,-1], axis=0)

# Depict illustration
fig = plt.figure()
plt.streamplot(X,Y,Ex,Ey, start_points=stream_points, density=5, linewidth=None, color='black')
plt.axhline(0,color='black',linestyle=':')
plt.gca().set_aspect('equal')

fig.set_figheight(15*cm)
plt.title(r'End Effector Movement Path', pad=10)
plt.xlabel(r'Horizontal Distance From Target (mm)', fontsize=normalsize-1)
plt.ylabel(r'Vertical Distance From Target (mm)', fontsize=normalsize-1)

# plt.grid()
plt.show()
plt.savefig("foot_path.pdf", format="pdf", bbox_inches="tight")


/home/epep/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in true_divide
/home/epep/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in true_divide
/home/epep/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply
/home/epep/.local/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in multiply
